In [2]:
from oct_Utils import *
root_path = '/home/octusr3/project/oct/output'
exp_name = '1102/disc_num_r50_512'
df = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/data.csv')

/home/octusr3/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


df

df

In [3]:
valid_mask ='''[[0 0 0 0 0 0 0 0 0 0]
                [0 0 0 1 1 1 1 0 0 0]
                [0 0 1 1 1 1 1 1 0 0]
                [0 1 1 1 1 1 1 1 1 0]
                [1 1 1 1 1 1 1 1 1 0]
                [1 1 1 1 1 1 1 1 1 0]
                [0 1 1 1 1 1 1 1 1 0]
                [0 0 1 1 1 1 1 1 0 0]
                [0 0 0 1 1 1 1 0 0 0]
                [0 0 0 0 0 0 0 0 0 0]]'''

od_valid_mask = str_to_np_mat(valid_mask) == 1


def restore_pred_matrix(row):
    result = np.zeros((10, 10), dtype=float)
    result[od_valid_mask] = str_to_np_array(row['pred'])#相当于把valid_point的地方,也就是矩阵为True的地方，赋值给pred,按照一行一行的顺序排列数据。
    if row['eye'] == 'OS':
        result = np.flip(result, axis=1)
    return result

def restore_pred_list(row):
    mat = restore_pred_matrix(row)
    if row['eye'] == 'OS':
        mat = np.flip(mat, axis=1)
    return mat[od_valid_mask].tolist()

def convert_num_to_list(row):
    mat = str_to_np_mat(row['num'])
    if row['eye'] == 'OS':
        mat = np.flip(mat, axis=1)
    return mat[od_valid_mask].tolist()

df['pred_mat'] = df.apply(lambda row: restore_pred_matrix(row), axis=1)
df['pred_array'] = df.apply(lambda row: restore_pred_list(row), axis=1)
df['num_array'] = df.apply(lambda row:convert_num_to_list(row), axis=1)

#print_df(df)

In [6]:
meta_df = pd.read_csv('/home/octusr3/project/oct/csv/0916.csv')
meta_df = meta_df.drop_duplicates(subset=['image_path'])#删除image_path的重复元素，删除掉重复的image_path
meta_df = meta_df[['image_path', 'age', 'floss', 'FP', 'FN']]
meta_df['age'] = pd.to_numeric(meta_df['age'],errors='coerce')
#print_df(meta_df)

join_df = df.merge(meta_df, on='image_path')
join_df = join_df.dropna()

#print_df(join_df)

In [8]:
meta_df

,image_path,age,floss,FP,FN
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143...,NaN,0.000000,0.01,0.00
1,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143...,NaN,0.000000,0.01,0.00
2,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-219/OD/O...,30.0,0.000000,0.01,0.00
3,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-219/OD/O...,30.0,0.000000,0.01,0.00
4,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-219/OD/O...,30.0,0.000000,0.01,0.00
...,...,...,...,...,...
26448,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,55.0,0.125000,0.01,0.05
26449,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,55.0,0.133333,0.05,0.05
26450,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,55.0,0.133333,0.05,0.05
26451,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,55.0,0.133333,0.05,0.05


In [7]:
join_df

,image_path,source,pid,dates,eye,pos,num,pd_prob,dataset,pred,pred_mat,pred_array,num_array,age,floss,FP,FN
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0,[23.822277 23.822277 23.603832 23.603832 23.60...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[23.822277, 23.822277, 23.603832, 23.603832, 2...","[34.0, 29.0, 31.0, 26.0, 31.0, 32.0, 31.0, 30....",42.0,0.071429,0.02,0.00
1,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0,[25.10477 25.10477 25.11046 25.11046 25.11...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.10477, 25.10477, 25.11046, 25.11046, 25.11...","[25.0, 28.0, 27.0, 29.0, 28.0, 27.0, 27.0, 30....",42.0,0.071429,0.02,0.04
2,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-201/2019...,CASIA数据汇总-new,CS-201,2019-06-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 3...,0,[25.115953 25.115953 25.11262 25.11262 25.11...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.115953, 25.115953, 25.11262, 25.11262, 25....","[19.0, 23.0, 23.0, 22.0, 25.0, 25.0, 27.0, 26....",54.0,0.000000,0.00,0.00
3,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-201/2019...,CASIA数据汇总-new,CS-201,2019-06-13,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0,[24.63522 24.63522 22.276436 22.276436 22.27...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[24.63522, 24.63522, 22.276436, 22.276436, 22....","[26.0, 27.0, 25.0, 24.0, 26.0, 27.0, 25.0, 28....",54.0,0.153846,0.00,0.00
4,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-204/2019...,CASIA数据汇总-new,CS-204,2019-06-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,0,[25.817066 25.817066 25.095592 25.095592 25.09...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.817066, 25.817066, 25.095592, 25.095592, 2...","[32.0, 31.0, 31.0, 32.0, 31.0, 27.0, 32.0, 32....",54.0,0.071429,0.00,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13387,20210820-sysu-oct/MAGIC4/MAGIC2238/20210628/OS...,MAGIC4,MAGIC2238,2021-06-28,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 1...,4,[17.569643 17.569643 27.657059 27.657059 27.65...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[17.569643, 17.569643, 27.657059, 27.657059, 2...","[25.0, 29.0, 26.0, 28.0, 30.0, 31.0, 29.0, 30....",64.0,0.000000,0.02,0.03
13388,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4,[18.342037 18.342037 28.247128 28.247128 28.24...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[18.342037, 18.342037, 28.247128, 28.247128, 2...","[18.0, 28.0, 20.0, 22.0, 30.0, 30.0, 31.0, 27....",55.0,0.125000,0.01,0.05
13389,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OD...,MAGIC4,MAGIC2242,2021-07-13,OD,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 4...,4,[17.592684 17.592684 30.116194 30.116194 30.11...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[17.592684, 17.592684, 30.116194, 30.116194, 3...","[18.0, 28.0, 20.0, 22.0, 30.0, 30.0, 31.0, 27....",55.0,0.125000,0.01,0.05
13390,20210820-sysu-oct/MAGIC4/MAGIC2242/20210713/OS...,MAGIC4,MAGIC2242,2021-07-13,OS,disc,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0. 3...,4,[18.456882 18.456882 23.74429 23.74429 23.74...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[18.456882, 18.456882, 23.74429, 23.74429, 23....","[32.0, 30.0, 27.0, 19.0, 30.0, 31.0, 30.0, 30....",55.0,0.133333,0.05,0.05


In [24]:
def extract_list_for_R(df, col):
    df = df.copy()
    df = df[['image_path', 'eye', 'dates', 'age', 'FP', 'FN', 'floss', col]]
    df = df.rename(columns={
        'image_path': 'id', 
        'dates': 'date', 
        'FP': 'fpr', 
        'FN': 'fnr', 
        'floss': 'fl'
    })
    df[[f'l{i}' for i in range(1, 55)]] = pd.DataFrame(df[col].tolist(), index= df.index)#这个比melt好用多了，melt融化数据很麻烦，非常复杂，弄得很复杂，最后还不如直接tolist() 构建新的数据集再join
    df = df.drop(columns=[col])
    df['time'] = '00:00:00'
    df['type'] = 'pwg'
    df['duration'] = '00:06:00'
    df = df[['id','eye','date','time','age','type','fpr','fnr','fl','duration'] + [f'l{i}' for i in range(1, 55)]]
    return df

df4R = extract_list_for_R(join_df, 'num_array')
df4R.to_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num.csv', index=False)

df4R = extract_list_for_R(join_df, 'pred_array')
df4R.to_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred.csv', index=False)

# 指标计算

## MAE

In [26]:
from sklearn.metrics import precision_recall_fscore_support

def get_metrics(exp_name, name):
    valid_cols = [f'l{i}' for i in range(1, 55)]
    valid_cols.remove('l26')
    valid_cols.remove('l35')
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred.csv')
    num = df_num[valid_cols].values
    pred = df_pred[valid_cols].values
    mae = mean_absolute_error(num, pred)
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num_pdp.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred_pdp.csv')
    
    class_mapping = {
        0: 1,
        0.005: 1,
        0.01: 1,
        0.02: 1,
        0.05: 1,
        0.95: 0,
        0.98: 0,
        0.99: 0,
        0.995: 0,
        1: 0,
    }
    for col in valid_cols:
        df_num[col] = df_num[col].map(class_mapping)
        df_pred[col] = df_pred[col].map(class_mapping)

    y_true = df_num[valid_cols].values.reshape(-1)
    y_pred = df_pred[valid_cols].values.reshape(-1)

    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]

    acc = (TP + TN) / (TP + FP + FN + TN)
    sensitivity = TP / (TP + FN)
    specificity = TN / (FP + TN)
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num_pdp.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred_pdp.csv')
    
    class_mapping = {
        0: 4,
        0.005: 4,
        0.01: 3,
        0.02: 2,
        0.05: 1,
        0.95: 0,
        0.98: 0,
        0.99: 0,
        0.995: 0,
        1: 0,
    }
    for col in valid_cols:
        df_num[col] = df_num[col].map(class_mapping)
        df_pred[col] = df_pred[col].map(class_mapping)

    y_true = df_num[valid_cols].values.reshape(-1)
    y_pred = df_pred[valid_cols].values.reshape(-1)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    
    _, _, fscores, _ = precision_recall_fscore_support(y_true, y_pred)
    return {
        'exp name': name,
        
        ('Regression', 'mae'): f'{mae:.3f}',
        
        ('2-class', 'accuracy'): f'{acc:.3f}',
        ('2-class', 'sensitivity'): f'{sensitivity:.3f}',
        ('2-class', 'specificity'): f'{specificity:.3f}',
        ('2-class', 'precision'): f'{precision:.3f}',
        ('2-class', 'recall'): f'{recall:.3f}',
        ('2-class', 'f1'): f'{F1:.3f}',
        
        ('5-class', '0.95-f1'): f'{fscores[0]:.3f}',
        ('5-class', '0.05-f1'): f'{fscores[1]:.3f}',
        ('5-class', '0.02-f1'): f'{fscores[2]:.3f}',
        ('5-class', '0.01-f1'): f'{fscores[3]:.3f}',
        ('5-class', '0.005-f1'): f'{fscores[4]:.3f}',
        
        ('5-class', 'macro-f1'): f'{macro_f1:.3f}',
        ('5-class', 'micro-f1'): f'{micro_f1:.3f}',
    }
df_stat = []
df_stat.append(get_metrics('1005/macula_num_r50_380', 'Macula'))
df_stat.append(get_metrics('1102/disc_num_r50_512', 'Disc'))

keys = [('Regression', 'mae'),
        ('2-class', 'accuracy'),
        ('2-class', 'sensitivity'),
        ('2-class', 'specificity'),
        ('2-class', 'precision'),
        ('2-class', 'recall'),
        ('2-class', 'f1'),
        
        ('5-class', '0.95-f1'),
        ('5-class', '0.05-f1'),
        ('5-class', '0.02-f1'),
        ('5-class', '0.01-f1'),
        ('5-class', '0.005-f1'),
        
        ('5-class', 'macro-f1'),
        ('5-class', 'micro-f1')]
df_stat = pd.DataFrame(df_stat, index=['Macula', 'Disc'], columns=pd.MultiIndex.from_tuples(keys))
#df_stat = df_stat.set_index('exp name')
#print_df(df_stat)

In [28]:
df_stat

Regression  2-class                                                  \
              mae accuracy sensitivity specificity precision recall     f1   
Macula      4.250    0.639       0.374       0.680     0.155  0.374  0.220   
Disc        3.238    0.812       0.080       0.931     0.159  0.080  0.106   

       5-class                                                     
       0.95-f1 0.05-f1 0.02-f1 0.01-f1 0.005-f1 macro-f1 micro-f1  
Macula   0.765   0.054   0.023   0.023    0.145    0.202    0.612  
Disc     0.895   0.039   0.007   0.007    0.029    0.195    0.804